<a href="https://colab.research.google.com/github/Harshil323/Sementic-Code-Search/blob/main/SCS_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# initially we are connect to the google drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


## Grab the data and pre-process it

In [ ]:
# importing all the necessary files
import ast
import sqlite3
import glob
import re
from pathlib import Path
import astor
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from pathlib import Path

In [ ]:
from pathlib import Path
import ast
import traceback
# directory name
dirname = '/content/gdrive/MyDrive/Dataset/'

# create a dataframe for dataset
df = pd.DataFrame()
df["Source_Code"] = "Source_code"
df["repo_path"] = "repo_path"
# giving directory name to Path() function
paths = Path(dirname).glob('**/*.py',)

# iterating over all files
for path in paths:
    txt = path.read_text()
    # print(type(txt))
    # print(path)
    # Check whether the systax of the python file is true or not
    valid = True
    try:
        ast.parse(txt)
    except SyntaxError:
        valid = False
        traceback.print_exc()
        print(path)
    # print(valid)
    if valid == True:
        df = df.append({'Source_code' : txt,'repo_path':path},ignore_index=True)

Traceback (most recent call last):
  File "<ipython-input-3-e815675ecf1e>", line 22, in <module>
    ast.parse(txt)
  File "/usr/lib/python3.7/ast.py", line 35, in parse
    return compile(source, filename, mode, PyCF_ONLY_AST)
  File "<fstring>", line 1
    (data = )
          ^
SyntaxError: invalid syntax


/content/gdrive/MyDrive/Dataset/3.py


### in the above example a file with compilation error is found for python so it will be excluded

In [ ]:
df.head()
df.drop(['Source_Code'],axis=1)

,repo_path,Source_code
0,/content/gdrive/MyDrive/Dataset/1.py,"# -*- coding: utf-8 -*-\n""""""\nCreated on Wed F..."
1,/content/gdrive/MyDrive/Dataset/2.py,"# -*- coding: utf-8 -*-\n""""""\nCreated on Thu F..."
2,/content/gdrive/MyDrive/Dataset/4.py,"# -*- coding: utf-8 -*-\n""""""\nCreated on Thu F..."
3,/content/gdrive/MyDrive/Dataset/5.py,"# -*- coding: utf-8 -*-\n""""""\nCreated on Thu F..."
4,/content/gdrive/MyDrive/Dataset/6.py,"# -*- coding: utf-8 -*-\n""""""\nCreated on Thu F..."
5,/content/gdrive/MyDrive/Dataset/7.py,"# -*- coding: utf-8 -*-\n""""""\nCreated on Thu F..."
6,/content/gdrive/MyDrive/Dataset/8.py,"# -*- coding: utf-8 -*-\n""""""\nCreated on Thu F..."
7,/content/gdrive/MyDrive/Dataset/9.py,"# -*- coding: utf-8 -*-\n""""""\nCreated on Thu F..."
8,/content/gdrive/MyDrive/Dataset/10.py,"# -*- coding: utf-8 -*-\n""""""\nCreated on Thu F..."
9,/content/gdrive/MyDrive/Dataset/11.py,"# -*- coding: utf-8 -*-\n""""""\nCreated on Thu F..."


In [ ]:
# now pre process the daataset
"""
1) we create the docstring and function code pair
2) extarct function name, function dcstring extarcted
3) tokenize docstring and function code
"""
def tokenize_docstring(text):
    """This is the function to tokenize the text"""

    # first wr parition the whole code
    before, keyword, after = text.partition(':')
    before, keyword, after = before.partition('@param')
    before, keyword, after = before.partition('param')
    before, keyword, after = before.partition('@brief')

    # all the words after will be tokenize
    if(after):
        words = RegexpTokenizer(r'[a-zA-Z0-9]+').tokenize(after)
    else:
    # all the docstring is tokenize
        before, keyword, after = before.partition('@')
        words = RegexpTokenizer(r'[a-zA-Z0-9]+').tokenize(before)

    # Every text is converted into lower case
    new_words= [word.lower() for word in words if word.isalnum()]

    return new_words

In [ ]:
def tokenize_code(text):
    """This is the function to tokenize the function code"""

    # fioorstly we remove all the unnecessary details such as decorators and function signature untill a function is found
    keyword = 'def '
    # partition the text after the 'def' afetr funding the function
    before, keyword, after = text.partition(keyword)
    # tokenize the function code
    words = RegexpTokenizer(r'[a-zA-Z0-9]+').tokenize(after)

    # lower casing the function code and remove the single alphabate variable
    new_words= [word.lower() for word in words if (word.isalpha() and len(word)>1) or (word.isnumeric())]
    return new_words

In [ ]:
def get_function_docstring_pairs(blob):

    """This is the function to create a list for function and docstring pair"""
    pairs = []
    try:
        # by using ast module we create AST structure of the python code
        module = ast.parse(blob)
        # get the class name using ast module
        classes = [node for node in module.body if isinstance(node, ast.ClassDef)]
        # get all the funtion name available in a python code
        functions = [node for node in module.body if isinstance(node, ast.FunctionDef)]
        # run through all the classes to find the result for
        for _class in classes:
            functions.extend([node for node in _class.body if isinstance(node, ast.FunctionDef)])
        # Now iterate through all the function to get the original function without comment and the etarct the docstring for further analysis
        for f in functions:
            # Conver the function to get the AST tree
            source = astor.to_source(f)
            # get the docstring from the function if available
            docstring = ast.get_docstring(f) if ast.get_docstring(f) else ''
            # now in the function remove the docstring and the comment
            function = source.replace(ast.get_docstring(f, clean=False), '') if docstring else source

            # Extracts function name, line number of the function in the source code, original function, function tokens and docstring tokens
            pairs.append((f.name,
                          f.lineno,
                          source,
                          ' '.join(tokenize_code(function)),
                          ' '.join(tokenize_docstring(docstring.split('\n\n')[0]))
                         ))
    except (AssertionError, MemoryError, SyntaxError, UnicodeEncodeError):
        pass
    return pairs

In [ ]:
def get_function_docstring_pairs_list(blob_list):
    """apply the function `get_function_docstring_pairs` on a list of blobs"""
    return [get_function_docstring_pairs(b) for b in blob_list]

In [ ]:
func_doc = get_function_docstring_pairs_list(df.Source_code.tolist())

In [ ]:
print(len(func_doc))

31


In [ ]:
df['pairs'] = func_doc

In [ ]:
df = df.set_index(['repo_path'])['pairs'].apply(pd.Series).stack()
df = df.reset_index()
df.columns = ['repo_path', '_', 'pair']

# now separate the function and docstring pair
df['f_name'] = df['pair'].apply(lambda p: p[0])
df['line_number'] = df['pair'].apply(lambda p: p[1])
df['original'] = df['pair'].apply(lambda p: p[2])
df['f_tokens'] = df['pair'].apply(lambda p: p[3])
df['d_tokens'] = df['pair'].apply(lambda p: p[4])

In [ ]:
# now duplicate all the duplicates
before = len(df)
df = df.drop_duplicates(['f_tokens'])
after = len(df)
print(f'found and reove {before - after:,} duplicate rows')

NameError: ignored

In [ ]:
# now separate function with and without docstring from the dataset
# get the length of the docstring
def listlen(x):
    if not isinstance(x, list):
        return 0
    return len(x)
with_= df[df.d_tokens.str.split().apply(listlen) >= 3]
without_ = df[df.d_tokens.str.split().apply(listlen) < 3]
print('Function with docstring',len(with_))
print('Funtion without docstring',len(without_))

Function with docstring 42
Funtion without docstring 9


In [ ]:
# Save the final dataset
with_.to_csv('/content/gdrive/MyDrive/SCE_dataset/processed_full.csv')

In [ ]:
train, valid = train_test_split(with_, train_size=0.7, random_state=8081)
train, test = train_test_split(train, train_size=0.9, random_state=8081)

In [ ]:
train.to_csv("/content/gdrive/MyDrive/SCE_dataset/train.csv")
valid.to_csv("/content/gdrive/MyDrive/SCE_dataset/validation.csv")
test.to_csv("/content/gdrive/MyDrive/SCE_dataset/test.csv")

## Convert Docstring to a 768 dimension vector

In [ ]:
# now first install all the required libraries
# the albert transformer is trained over tensorflow 1.15.0
# we install tensorflow, tranformer, sentencepiece libraries
!pip install transformers
!pip install albert-tensorflow
!pip install sentencepiece


     |████████████████████████████████| 4.0 MB 12.0 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
     |████████████████████████████████| 6.5 MB 40.4 MB/s 
     |████████████████████████████████| 895 kB 43.7 MB/s 
     |████████████████████████████████| 596 kB 50.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 81 kB 5.8 MB/s 
     |████████████████████████████████| 1.2 MB 12.3 MB/s 


In [ ]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3 MB 27 kB/s 
     |████████████████████████████████| 503 kB 46.2 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
     |████████████████████████████████| 3.8 MB 37.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=7bf59d50193f60a5d9ff9f0baa4c06cec19b29fb68cf0a3c40e3288c79a486e2
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [ ]:
!python "/content/gdrive/MyDrive/Albert model/albert-master/albert-master/create_pretraining_data.py" \
  --input_file  "/content/gdrive/MyDrive/SCE_dataset/docstrings.txt" \
  --output_file  "/content/gdrive/MyDrive/Albert_model_output_1/tf_example" \
  --vocab_file  "/content/gdrive/MyDrive/Albert model/albert_base_3/assets/30k-clean.vocab" \
  --spm_model_file "/content/gdrive/MyDrive/Albert model/albert_base_3/assets/30k-clean.model" \
  --do_lower_case True \
  --max_seq_length 50 \
  --max_predictions_per_seq 8 \
  --random_seed 12345 \
  --dupe_factor 5


W0410 20:52:09.972674 140429856573312 module_wrapper.py:139] From /usr/local/lib/python3.7/dist-packages/albert/tokenization.py:240: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:loading sentence piece model
I0410 20:52:09.972921 140429856573312 tokenization.py:240] loading sentence piece model
INFO:tensorflow:*** Reading from input files ***
I0410 20:52:10.812790 140429856573312 create_pretraining_data.py:631] *** Reading from input files ***
INFO:tensorflow:  /content/gdrive/MyDrive/SCE_dataset/docstrings.txt
I0410 20:52:10.813139 140429856573312 create_pretraining_data.py:633]   /content/gdrive/MyDrive/SCE_dataset/docstrings.txt
INFO:tensorflow:number of instances: 31
I0410 20:52:11.222163 140429856573312 create_pretraining_data.py:641] number of instances: 31
INFO:tensorflow:*** Writing to output files ***
I0410 20:52:11.222572 140429856573312 create_pretraining_data.py:644] *** Writing to output files ***
INFO:tensorflow:  

In [ ]:
!python "/content/gdrive/MyDrive/Albert model/albert-master/albert-master/run_pretraining.py" \
    --input_file "/content/gdrive/MyDrive/Albert_model_output_1/tf_example" \
    --output_dir "/content/gdrive/MyDrive/Albert_model_output_1" \
    --albert_config_file "/content/gdrive/MyDrive/Albert model/albert_base_3/assets/albert_config.json" \
    --do_train \
    --do_eval \
    --train_batch_size=64 \
    --eval_batch_size=32 \
    --max_seq_length 50 \
    --max_predictions_per_seq 8 \
    --optimizer 'lamb' \
    --learning_rate .00176 \
    --num_train_steps 100 \
    --num_warmup_steps 10 \



W0410 20:52:19.295110 140400511838080 module_wrapper.py:139] From /usr/local/lib/python3.7/dist-packages/albert/modeling.py:116: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

INFO:tensorflow:*** Input Files ***
I0410 20:52:19.760158 140400511838080 run_pretraining.py:484] *** Input Files ***
INFO:tensorflow:  /content/gdrive/MyDrive/Albert_model_output_1/tf_example
I0410 20:52:19.760502 140400511838080 run_pretraining.py:486]   /content/gdrive/MyDrive/Albert_model_output_1/tf_example
W0410 20:52:19.761241 140400511838080 estimator.py:1994] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fb11b3d0d40>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': '/content/gdrive/MyDrive/Albert_model_output_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_opt

In [ ]:
import logging
import torch
from transformers import AlbertConfig, AlbertForPreTraining, load_tf_weights_in_albert


logging.basicConfig(level=logging.INFO)


def convert_tf_checkpoint_to_pytorch(tf_checkpoint_path, albert_config_file, pytorch_dump_path):
    # Initialise PyTorch model
    config = AlbertConfig.from_json_file(albert_config_file)
    print("Building PyTorch model from configuration: {}".format(str(config)))
    model = AlbertForPreTraining(config)

    # Load weights from tf checkpoint
    load_tf_weights_in_albert(model, config, tf_checkpoint_path)

    # Save pytorch-model
    print("Save PyTorch model to {}".format(pytorch_dump_path))
    torch.save(model.state_dict(), pytorch_dump_path)



convert_tf_checkpoint_to_pytorch('/content/gdrive/MyDrive/Albert_model_output_1/model.ckpt-100.index','/content/gdrive/MyDrive/Albert model/albert_base_3/assets/albert_config.json' , '/content/gdrive/MyDrive/Albert_model_output_1/pytorch_model.bin')

Building PyTorch model from configuration: AlbertConfig {
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "vocab_size": 30000
}

Save PyTorch model to /content/gdrive/MyDrive/Albert_model_output_1/pytorch_model.bin


In [ ]:
!pip install tensorflow==2.8.0

     |████████████████████████████████| 497.5 MB 26 kB/s 
     |████████████████████████████████| 5.8 MB 38.3 MB/s 
     |████████████████████████████████| 462 kB 48.9 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [ ]:
from transformers import  AlbertConfig,TFAlbertModel
from transformers import AlbertTokenizer
from transformers import  AlbertConfig,TFAlbertModel
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

In [ ]:
config = AlbertConfig.from_pretrained('/content/gdrive/MyDrive/Config_and_model', output_hidden_states=True)

model = TFAlbertModel.from_pretrained('/content/gdrive/MyDrive/Config_and_model', config=config,  from_pt=True)
print(model.config)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.decoder.bias', 'albert.embeddings.position_ids', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on,

AlbertConfig {
  "_name_or_path": "/content/gdrive/MyDrive/Config_and_model",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "vocab_size": 30000
}



In [ ]:
import pandas as pd
import tensorflow as tf

# now load the training, validation and testing from the drive
train_df = pd.read_csv("/content/gdrive/MyDrive/SCE_dataset/train.csv")
print(len(train_df['d_tokens'].values))

26


In [ ]:
# by using the Albert tokenization and trained albert model the docstring is embedded into 768 dimension vector
avg_embeddings = []
for count,item in enumerate(train_df['d_tokens'].values):
    e = tokenizer.encode(item, max_length=512)
    input = tf.constant(e)[None, :]
    output = model(input)
    v = [0]*768
    for i in range(1, len(input[0])-1):
        v = v + output[0][0][i].numpy()
    avg_embeddings.append(v/len(input[0]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
print(len(avg_embeddings))
# output confirms that all the docstring are converted into 768 dimension vector

26


In [ ]:
# now save the embedding into tsv file for further use
# Save the sentence embeddings in a .tsv file
import csv
with open("/content/gdrive/MyDrive/SCE_dataset/docstrin_embeddings.tsv","w+",newline='') as my_csv:
    csvWriter = csv.writer(my_csv,delimiter='\t')
    csvWriter.writerows(avg_embeddings)